# Сверточные нейронные сети для анализа текста

In [10]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 8.2 MB 24.0 MB/s 


In [12]:
!pip install stop-words

  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=aae7f6acdc0014ef4e38b2814e0c1d6f1c1d07e8673b43ea0a1aa7c66e200ef9
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [23]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist


In [36]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

import tensorflow as tf

In [19]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
df = pd.read_excel('../content/drive/MyDrive/отзывы за лето.xls')

In [3]:
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [7]:
df.drop('Date', axis=1, inplace=True)

In [8]:
df_train, for_test = train_test_split(df, test_size=0.35,  shuffle=True)
df_test, df_val = train_test_split(for_test, test_size=0.42, shuffle=True)

In [9]:
df_train.head(5)

,Rating,Content
11946,5,Очень удобное и простое в обращении приложение!
17915,3,Почему нельзя пополнить счёт другого номера .в...
3186,5,"Всё очень удобно сделано,мне нравится."
16647,5,"Приложение работает не плохо , все устраевает"
2939,5,Елена


In [27]:
max_words = 2000 
max_len = 20
num_classes = 6

In [14]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

In [15]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [17]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [21]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [22]:
tokens = word_tokenize(train_corpus)

In [25]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [28]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [29]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [30]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [32]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

In [33]:
x_train.shape, max_len

((13428, 20), 20)

In [37]:
y_train = tf.keras.utils.to_categorical(df_train["Rating"], num_classes)
y_val = tf.keras.utils.to_categorical(df_val["Rating"], num_classes)

In [38]:
epochs = 20
batch_size = 16
print_batch_n = 100

In [39]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(num_classes))
model.add(Activation('softmax'))


model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [40]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

In [41]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
756/756 [==============================] - 21s 9ms/step - loss: 0.8291 - accuracy: 0.7365 - val_loss: 0.6081 - val_accuracy: 0.7766


In [42]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

190/190 [==============================] - 0s 2ms/step - loss: 0.6684 - accuracy: 0.7676


Test score: 0.6684021949768066
Test accuracy: 0.7676102519035339


In [43]:
def different_convalutions(conv=(2,2)):
    results = []
    for temp_conv in range(conv[0], conv[1]+1):
        model = Sequential()
        model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
        model.add(Conv1D(128, temp_conv))
        model.add(Activation("relu"))
        model.add(GlobalMaxPool1D())
        model.add(Activation("relu"))
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))

        model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
        early_stopping=EarlyStopping(monitor='val_loss')  


        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_split=0.1,
                            callbacks=[tensorboard, early_stopping])
        # results.append((history.history, temp_conv))
        results.append(history.history)
    return results, [i for i in range(conv[0], conv[1]+1)]

In [44]:
res = different_convalutions((1,15))

Epoch 1/20
756/756 [==============================] - 5s 6ms/step - loss: 0.9008 - accuracy: 0.7157 - val_loss: 0.6079 - val_accuracy: 0.7856
Epoch 1/20
756/756 [==============================] - 6s 6ms/step - loss: 0.8653 - accuracy: 0.7271 - val_loss: 0.6196 - val_accuracy: 0.7759
Epoch 1/20
756/756 [==============================] - 6s 7ms/step - loss: 0.8518 - accuracy: 0.7330 - val_loss: 0.6020 - val_accuracy: 0.7870
Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.8582 - accuracy: 0.7344 - val_loss: 0.6144 - val_accuracy: 0.7870
Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.8375 - accuracy: 0.7353 - val_loss: 0.6019 - val_accuracy: 0.7856
Epoch 1/20
756/756 [==============================] - 8s 9ms/step - loss: 0.8199 - accuracy: 0.7435 - val_loss: 0.6089 - val_accuracy: 0.7848
Epoch 1/20
756/756 [==============================] - 8s 9ms/step - loss: 0.8222 - accuracy: 0.7410 - val_loss: 0.6073 - val_accuracy: 0.7848
Epoch 

In [45]:
pd.DataFrame(res[0], index=res[1])

,loss,accuracy,val_loss,val_accuracy
1,[0.7288829684257507],[0.7559784650802612],[0.60785973072052],[0.7855547070503235]
2,[0.7271360158920288],[0.7567232251167297],[0.6196072101593018],[0.7758749127388]
3,[0.7232279777526855],[0.7583781480789185],[0.6020014882087708],[0.7870439291000366]
4,[0.7205296158790588],[0.7635912299156189],[0.6143738627433777],[0.7870439291000366]
5,[0.7180187702178955],[0.7603641152381897],[0.6019331812858582],[0.7855547070503235]
6,[0.7108978033065796],[0.7636739611625671],[0.6088784337043762],[0.7848101258277893]
7,[0.7122489213943481],[0.7648324370384216],[0.6073293685913086],[0.7848101258277893]
8,[0.7124065160751343],[0.7625982761383057],[0.6035803556442261],[0.7810871005058289]
9,[0.7168221473693848],[0.7609432935714722],[0.6091580390930176],[0.7870439291000366]
10,[0.7168812155723572],[0.7611915469169617],[0.6184477806091309],[0.7803425192832947]


In [47]:
with open('../content/drive/MyDrive/model.txt', 'r', encoding='utf8') as f:
    pretrained_words = f.readlines()
pretrained_words.pop(0)

'189193 300\n'

In [48]:
pretrained_words[1].split()[0].split('_')[0]

'быть'

In [49]:
np.asarray(pretrained_words[1].split()[1:], dtype='float32').shape

(300,)

In [50]:
counter = 0
embeddings_index = {}
for pre_word in pretrained_words:
    counter += 1
    word = pre_word.split()[0].split('_')[0]
    coefs = np.asarray(pre_word.split()[1:], dtype='float32')
    embeddings_index[word] = coefs
print(counter)

189193


In [51]:
EMBEDDING_DIM = 300

In [52]:

embedding_matrix = np.zeros((len(vocabulary) + 1, EMBEDDING_DIM))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [53]:
embedding_layer = Embedding(input_dim=len(vocabulary) + 1,
                            output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [54]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(50))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 20, 300)           600000    
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 16, 128)           192128    
_________________________________________________________________
activation_47 (Activation)   (None, 16, 128)           0         
_________________________________________________________________
global_max_pooling1d_16 (Glo (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 50)                6450      
_________________________________________________________________
activation_48 (Activation)   (None, 50)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 6)               

In [55]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [56]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.9298 - accuracy: 0.7198 - val_loss: 0.7065 - val_accuracy: 0.7424


**Вывод:**
Результат модели с предобученным эмбеддингом оказался чуть ниже. Веорятно, это связано с безграмотностью отзывов (присутствуют смайлы) - в претренированном эмбеддинге таких слов просто нет